In [1]:
from DistilSR import *
import numpy as np
from tqdm import tqdm
import pandas as pd
from scipy.optimize import minimize
import itertools
from sklearn.model_selection import train_test_split
from pmlb import fetch_data, regression_dataset_names
from collections.abc import Iterable
import random
import warnings
warnings.filterwarnings("ignore")

exp_set = get_exp_set(3)
random.seed(0)
synthetic_list = random.choices(sorted([i for i in list(exp_set) if i.count("R")==4]), k=10)

In [2]:
def cost(x, xdata, ydata, lambda_string):
    y_pred = eval(lambda_string)(x, xdata)
    return np.mean(((y_pred - ydata))**2) 

def all_cost(x, xdata, ydata, lambda_string):
    y_pred = eval(lambda_string)(x, xdata)
    if lambda_string.count("xdata")==1 or not isinstance(y_pred, Iterable):
        y_pred = np.array([y_pred for i in range(len(ydata))])
    y_pred = np.array([min(max(i,-9999),9999) for i in y_pred])
    return [np.mean(np.abs(y_pred - ydata)), np.mean(((y_pred - ydata))**2)]

def random_constant():
    return np.random.randint(1, 31)/10

def MSR(opt_type="BFGS",dataset_name=0,random_state=0):
    synthetic_eq = synthetic_list[dataset_name]
    synthetic_var_count = synthetic_eq.count("R")
    random.seed(0)
    np.random.seed(0)
    const_index = 3
    random.seed(random_state)
    np.random.seed(random_state)
    X = np.random.randint(1, 31, size=(100,synthetic_var_count-1))/10
    idx = 0
    for i in range(synthetic_var_count):
        if i==const_index:
            synthetic_eq = synthetic_eq.replace("R",f"{np.random.rand(1)*2+1}", 1)
        else:
            synthetic_eq = synthetic_eq.replace("R",f"X[:,{idx}]", 1)
            idx+=1
    y = eval(synthetic_eq)
    y = np.array([min(max(i,-9999),9999) for i in y])
    print(f"GROUND TRUTH: {synthetic_eq}")
    train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=random_state,test_size=0.25)
    shuffle_idx = np.random.permutation(len(train_y))
    train_X = train_X[shuffle_idx]
    train_y = train_y[shuffle_idx]
    master_list=[]
    xdata = np.array(train_X).T
    ydata = np.array(train_y)
    xvalid = np.array(test_X).T
    yvaliddata= np.array(test_y)
    num_of_feature = xdata.shape[0]
    for i in range(1):
        mse_tuple = tuple()
        for test_eq in tqdm(exp_set):
            test_eq_orig = test_eq
            R_count = test_eq.count("R")
            lambda_string = "lambda x,xdata:"
            possible_combi_of_num = np.array(
                list(itertools.product(range(num_of_feature + 1), repeat=R_count))
            )
            for combi_var in itertools.product(range(num_of_feature+1), repeat=R_count):
                lambda_string = "lambda x,xdata:"
                test_eq=test_eq_orig
                index = 0
                for i in combi_var:
                    if i==num_of_feature:
                        test_eq = test_eq.replace("R", f"x[{index}]", 1)
                        index+=1
                    else:
                        test_eq = test_eq.replace("R", f"xdata[{i}]", 1)
                lambda_string += test_eq
                if index<1:
                    index = 1
                try:
                    res = minimize(cost,
                                   x0 = [random_constant() for i in range(index)],
                                   args=(xdata,ydata,lambda_string),
                                   method=opt_type,options={"maxiter":150})
                    optimized_cost = cost(res.x, xdata, ydata, lambda_string)
                    all_cost_list = all_cost(res.x, xvalid, yvaliddata, lambda_string)
                    master_list.append((optimized_cost,test_eq_orig,test_eq,res.x,res.nit,*all_cost_list))
                except RuntimeError:
                    print("No fit found")
    return master_list

In [3]:
dataset_name = 0 #S1
for rand_num in range(3):
    pd.DataFrame(MSR(dataset_name=dataset_name,random_state=rand_num)).to_csv(f'{dataset_name}_{rand_num}_excel.csv')
print("Dataset-(100,100,100)-S1-1")
consolidated_df = pd.read_csv('Dataset-(100,100,100)-S1-1.csv')
for rand_num_1 in range(1):
    for rand_num_2 in range(3):
        rand_num = rand_num_1*3+rand_num_2
        df = pd.read_csv(f'{dataset_name}_{rand_num}_excel.csv')
        if rand_num_2==0:
            master_df = df.copy()
        master_df[f"MSE_{rand_num_2}"] = df["0"]
        master_df[f"MSE_{rand_num_2}_TEST"] = df["6"]
    master_df["MSE_f1_TRAIN"] = 3/(1/master_df["MSE_0"]+1/master_df["MSE_1"]+1/master_df["MSE_2"])
    master_df["MSE_AVERAGE_TEST"] = (master_df["MSE_0_TEST"]+master_df["MSE_1_TEST"]+master_df["MSE_2_TEST"])/3
    print(f'Top Recovered Equation by MSR: {master_df.iloc[master_df["MSE_f1_TRAIN"].idxmin()][3]}')
    print(f'TEST NRMSE: {np.sqrt(master_df["MSE_AVERAGE_TEST"][master_df["MSE_f1_TRAIN"].idxmin()])/np.std(consolidated_df["y"])}')

GROUND TRUTH: Sub(Div(X[:,0],X[:,1]),Pow(X[:,2],[1.13833399]))


100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [04:09<00:00,  1.65s/it]


GROUND TRUTH: Sub(Div(X[:,0],X[:,1]),Pow(X[:,2],[2.8774234]))


100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [05:19<00:00,  2.11s/it]


GROUND TRUTH: Sub(Div(X[:,0],X[:,1]),Pow(X[:,2],[1.86477819]))


100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [04:56<00:00,  1.97s/it]


Dataset-(100,100,100)-S1-1
Top Recovered Equation by MSR: Sub(Div(xdata[0],xdata[1]),Pow(xdata[2],x[0]))
TEST NRMSE: 6.2914323249391096e-09
